## 1) Importing dependencies

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input,decode_predictions
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model,Sequential
from os import listdir
from os.path import isfile, join

## 2) Setting paths to image folders

In [2]:
main_path='/path'   # Change this to path where suitable folders have been made
train_path=main_path+'/train'
val_path=main_path+'/val'

## 3) Defining image-data generators

In [4]:
img_height, img_width=(224,224)
ImageDataGenerator=tf.keras.preprocessing.image.ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,
                                 height_shift_range=0.2,
                                 width_shift_range=0.2,
                                 rotation_range=40,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                 fill_mode='nearest')
train_generator=train_datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical')
val_generator=train_datagen.flow_from_directory(
    val_path,
    target_size=(img_height, img_width),
    batch_size=40,
    class_mode='categorical')

Found 801 images belonging to 10 classes.
Found 40 images belonging to 10 classes.


## 4) Building model using pre-trained Resnet50 with few custom layers as well

In [5]:
base_model=ResNet50(include_top=False,input_shape=(224,224,3),weights='imagenet')
x=base_model.output
x=tf.keras.layers.GlobalAveragePooling2D()(x)
x=tf.keras.layers.Dense(1024,activation='relu')(x)
x=tf.keras.layers.Dense(512,activation='relu')(x)
predictions=tf.keras.layers.Dense(10,activation='softmax')(x)
model=Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable=False

94773248/94765736 [==============================] - 1s 0us/step


In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

## 6) Compiling model and training

In [7]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
history= model.fit(train_generator,validation_data=val_generator,epochs=20,verbose=1)

Epoch 1/20
26/26 [==============================] - 229s 8s/step - loss: 2.5503 - accuracy: 0.0974 - val_loss: 2.2216 - val_accuracy: 0.2000
Epoch 2/20
26/26 [==============================] - 12s 471ms/step - loss: 2.3242 - accuracy: 0.1398 - val_loss: 2.2730 - val_accuracy: 0.2250
Epoch 3/20
26/26 [==============================] - 12s 457ms/step - loss: 2.2976 - accuracy: 0.1473 - val_loss: 2.3679 - val_accuracy: 0.0500
Epoch 4/20
26/26 [==============================] - 12s 444ms/step - loss: 2.2908 - accuracy: 0.1261 - val_loss: 2.2479 - val_accuracy: 0.1750
Epoch 5/20
26/26 [==============================] - 12s 440ms/step - loss: 2.3021 - accuracy: 0.1261 - val_loss: 2.2698 - val_accuracy: 0.1750
Epoch 6/20
26/26 [==============================] - 12s 440ms/step - loss: 2.2934 - accuracy: 0.1174 - val_loss: 2.2393 - val_accuracy: 0.2500
Epoch 7/20
26/26 [==============================] - 11s 436ms/step - loss: 2.2645 - accuracy: 0.1548 - val_loss: 2.2384 - val_accuracy: 0.1750
E

## 7) Creating prediction score table for validation data

In [ ]:
# 'breeds' list should be same as the classes of data
breeds=['beagle', 'chihuahua', 'doberman', 'french_bulldog', 'golden_retriever', 
        'malamute', 'pug', 'saint_bernard', 'scottish_deerhound', 'tibetan_mastiff']
data=[]
true_label=[]
pred_label=[]
ids=[]
for breed in breeds:
    sub_path=val_path+'/'+breed
    file_names=[f for f in listdir(sub_path) if isfile(join(sub_path, f))]
    for name in file_names:
        img=plt.imread(sub_path+'/'+name)
        img=tf.image.resize(np.array([img]),[224,224])
        scores=model.predict(img)[0]
        data.append(scores)
        true_label.append(breed)
        ids.append(name[:-4])
        pred_label.append(breeds[np.argmax(scores)])
data=np.array(data)

df=pd.DataFrame(data,columns=breeds)
df['Image IDs']=ids
new_cols=['Image IDs']+list(df.columns[:-1])
df=df[new_cols]
df['Predicted label']=pred_label
df['True label']=true_label

## 8) Conclusion
Since there are only about 80 images per class for training, the accuracy is quite low.